In [ ]:
import pandas as pd
import numpy as np
from sklearn import *
import seaborn as sns
from imblearn.over_sampling import SMOTE 

## Load the data into environment

In [ ]:
train=pd.read_csv('D:\\DataScience\\AV_HackThon\\train.csv')
test=pd.read_csv('D:\\DataScience\\AV_HackThon\\test.csv')

In [ ]:
test.head()
# print ('train shape',train.shape)
# print('test shape',test.shape)

combine train and test variables into combined dataset so that, we can do feature extractions and feature transformations together.

In [ ]:
obj_cols =[col for col in train.columns if train[col].dtype=='object']
obj_cols

# test['origination_date'].unique()

except origination and first payment date other columns are not required, hence dropping them from the list

In [ ]:
obj_cols =list(set(obj_cols) - set(['source',
 'financial_institution','loan_purpose']))
obj_cols

In [ ]:
from datetime import datetime

# test['first_payment_date'].strftime("%d-%m-%Y")
 
test['first_payment_date']=test['first_payment_date'].apply(lambda x: datetime.strptime(x,"%b-%y"))
train['first_payment_date']=train['first_payment_date'].apply(lambda x: datetime.strptime(x,"%m/%Y"))
test['origination_date']=test['origination_date'].apply(lambda x: datetime.strptime(x,"%d/%m/%y"))
train['origination_date']=train['origination_date'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))

train['origination_date']=train['origination_date'].apply(lambda x: x.strftime("%d-%m-%Y"))
test['origination_date']=test['origination_date'].apply(lambda x: x.strftime("%d-%m-%Y"))
train['first_payment_date']=train['first_payment_date'].apply(lambda x: x.strftime("%d-%m-%Y"))
test['first_payment_date']=test['first_payment_date'].apply(lambda x: x.strftime("%d-%m-%Y"))


# test['origination_date'].apply(lambda x: datetime.strptime(x,"%d/%m/%y"))

In [ ]:
print(train['origination_date'].unique())
print(train['first_payment_date'].unique())
print(test['first_payment_date'].unique())
print(test['origination_date'].unique())

As you see from above cell, the unique values are less so it is better to use one hot encoding for these variables

In [ ]:
OH_cols_train.shape

one hot encoding for origination_date and first_payment_dateb

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!
onehotencoder=OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train  =pd.DataFrame(onehotencoder.fit_transform(train[obj_cols]))
OH_cols_valid = pd.DataFrame(onehotencoder.transform(test[obj_cols]))

OH_cols_train .index=train.index
OH_cols_valid.index=test.index

num_cols_train=[col for col in train.columns if train[col].dtype != 'object']
num_cols_test=[col for col in test.columns if test[col].dtype != 'object']

# loan id can't be using in trianing
num_cols_train.remove('loan_id') 
num_cols_test.remove('loan_id')

num_train=train[num_cols_train]
num_test=test[num_cols_test]

OH_train=pd.concat([num_train,OH_cols_train],axis=1)
OH_test=pd.concat([num_test,OH_cols_valid],axis=1)


In [ ]:
OH_train.columns

In [ ]:

sns.barplot(train['interest_rate'].round(),train['m13'],hue=train['number_of_borrowers'],ci=None)


In [ ]:
deliquency_cols2=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12','m13']
deliquency_cols=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']


# test['m']=test[deliquency_cols].sum(axis=1)
# test.head()

# test['m13']=test['m'].apply(lambda x: 1 if x>0 else 0 )

# test[['loan_id','m13']].to_csv('submission.csv',index=False)

In [ ]:
OH_train['m']=OH_train[deliquency_cols].sum(axis=1)
Last3_cols=['m10','m11','m12']
Last6_cols=['m9','m7','m8','m10','m11','m12']

OH_train['Last3']=OH_train[Last3_cols].sum(axis=1)
OH_train['Last6']=OH_train[Last6_cols].sum(axis=1)


In [ ]:
OH_test['m']=OH_test[deliquency_cols].sum(axis=1)
OH_test['Last3']=OH_test[Last3_cols].sum(axis=1)
OH_test['Last6']=OH_test[Last6_cols].sum(axis=1)

In [ ]:
train.corr()

In [ ]:
OH_test['credit_score']=OH_test['borrower_credit_score']+OH_test['co-borrower_credit_score']
OH_train['credit_score']=OH_train['borrower_credit_score']+OH_train['co-borrower_credit_score']

OH_test.drop(columns=['borrower_credit_score','co-borrower_credit_score'],inplace=True)
OH_train.drop(columns=['borrower_credit_score','co-borrower_credit_score'],inplace=True)

In [ ]:
deliquency_cols=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']
# OH_train['m']=OH_train[deliquency_cols].sum(axis=1)
# OH_test['m']=OH_test[deliquency_cols].sum(axis=1)

OH_test.drop(columns=deliquency_cols,inplace=True)
OH_train.drop(columns=deliquency_cols,inplace=True)
print(OH_train.columns)

In [ ]:
from sklearn.model_selection import train_test_split

cols=list(OH_train.columns)
cols.remove('m13')
X=OH_train.loc[:,cols]
y=OH_train.loc[:,'m13']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [ ]:
OH_train.groupby('m13').count()

As you see from above the output cell, this dataset is heavily imbalance, class 1 is undersampled, lets use some sampling methods to fix this

In [ ]:
from sklearn.utils import resample

X= pd.concat([X_train,y_train],axis=1)

default= X[X.m13==1]
no_default= X[X.m13==0]
no_default.reset_index(inplace=True)
print('count of default class',default.shape)
print('count of no default class',no_default.shape)


#up sample default class
default_upsampled=resample(default,replace=True,n_samples=len(no_default),random_state=10)
default_upsampled.reset_index(inplace=True)
default_upsampled.index

upsampled=pd.concat([default_upsampled,no_default],axis=0,ignore_index=True)

upsampled.shape

y_train=upsampled.m13
X_train=upsampled.drop(columns=['m13'])
X_train=X_train.drop(columns=['index'])

## under sample the data

In [ ]:

X= pd.concat([X_train,y_train],axis=1)

default= X[X.m13==1]
no_default= X[X.m13==0]
no_default.reset_index(inplace=True)

#up sample default class
no_default_undersample=resample(no_default,replace=True,n_samples=len(default),random_state=10)
undersampled=pd.concat([default,no_default_undersample],axis=0,ignore_index=True)

print('sample size',undersampled.shape)

y_train=undersampled.m13
X_train=undersampled.drop(columns=['m13'])
X_train=X_train.drop(columns=['index'])

print('y_train shape',y_train.shape)
print('x_train shape',X_train.shape)

## SMOTE : over sampling 

In [ ]:
from imblearn.over_sampling import SMOTE
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0) # recreate the train and test datasets

sm=SMOTE(random_state=12,ratio=1.0)
X_train_sm,y_train_sm=sm.fit_sample(X_train,y_train)

In [ ]:
# y_train=undersampled.m13
# undersampled.drop(columns=['m13','index'],inplace=True)
# X_train=undersampled.drop(columns=['m13'])
# X_train=X_train.drop(columns=['index'],inplace=True)
X_train=undersampled.copy()
X_train=X_train[cols]
y_train.head()
print(X_train.columns)
print(X_test.columns)


In [ ]:
def model_accuracy(model):
    model.fit(X_train,y_train)
    predict_test=model.predict(X_test)
    model_score=f1_score(y_test,predict_test)
    print('F1 score is ',f1_score(y_test,predict_test))
    
    return model_score

In [ ]:
def to_submit(model):
    predict_test=model.predict(OH_test.loc[:,cols])
    test['m13']=predict_test
    test.loc[:,['loan_id','m13']].to_csv('submission.csv',index=False)
    print('sumbmission file created')
    return None

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb

model_naiveBayes=MultinomialNB()
model_dt=DecisionTreeClassifier()
model_lr=LogisticRegression()
model_knn=KNeighborsClassifier(n_neighbors=10)
model_svm=SVC(kernel='linear',C=0.025,random_state=9)
model_RF=RandomForestClassifier()


In [ ]:
# print('leniar Regression',model_accuracy(model_lr))
# print('KNN Regression',model_accuracy(model_knn))
print('SVM Regression',model_accuracy(model_svm))
# print(X_train.shape)
# print('random Forest',model_accuracy(model_RF))
# to_submit(model_RF)

In [ ]:
params = {"objective":"reg:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10,'n_esitmators':100}

model_RF=xgb.XGBClassifier(missing=9999999999,
                objective='reg:logistic',
                max_depth =20,
                n_estimators=70,
                learning_rate=0.001, 
                nthread=5,
                subsample=1.0,
                colsample_bytree=0.7,
                min_child_weight = 5,
                importance_type='weight',
                eta=0.05,
                scale_pos_weight=1.70,
                reg_lambda=1.2,
                seed=1301)

model_accuracy(model_RF)
to_submit(model_RF)

In [ ]:
data_dmtraix=xgb.DMatrix(X_train,y_train)

params = {"objective":"reg:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmtraix, params=params, nfold=10,
                    num_boost_round=50,early_stopping_rounds=10,metrics=['auc'], as_pandas=True, seed=123)
cv_results
# print((cv_results["test-rmse-mean"]).tail(1))


In [ ]:
xgb_model = xgb.XGBClassifier()
print('Best number of trees = {}'.format(cv_results.shape[0]))
xgb_model.set_params(n_estimators=cv_results.shape[0])
print('Fit on the trainingsdata')
xgb_model.fit(X_train, y_train, eval_metric='auc')
print('Overall AUC:', metrics.roc_auc_score(y, xgb_model.predict_proba(X)[:,1]))
print('Predict the probabilities based on features in the test set')
pred = xgb_model.predict_proba(OH_test[cols], ntree_limit=cv_results.shape[0])



In [ ]:
pred_int = pred[:,1]>0.3
pred_int

submission = pd.DataFrame({"loan_id":test.loan_id, "m13":pred_int})
submission.to_csv("submission.csv", index=False)

In [ ]:
# print('NaiveBayes Score',model_accuracy(model_naiveBayes))
# print('Randomforest Score',model_accuracy(model_RF))
# print('Decision tree Score',model_accuracy(model_dt))
# print('Randomforest Score',model_accuracy(xgb_model))

to_submit(xgb_model)